In [1]:
import numpy as np

In [2]:
def generate_surface_points(center, radius, split_param=100):   
    u = np.linspace(0, 1, split_param)  
    v = np.linspace(0, 1, split_param)  

    u_grid, v_grid = np.meshgrid(u, v)

    u_spher = u_grid * 2 * np.pi  
    v_spher = np.arcsin(2 * v_grid - 1)  

    # Перевод в декартовы координаты
    x = center[0] + radius * np.cos(v_spher) * np.cos(u_spher)
    y = center[1] + radius * np.cos(v_spher) * np.sin(u_spher)
    z = center[2] + radius * np.sin(v_spher)

    decart_coords = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T

    normals = np.column_stack((x - center[0], y - center[1], z - center[2]))
    normals /= np.linalg.norm(normals, axis=1, keepdims=True)

    return decart_coords, normals


In [5]:
def generate_toroidal_surface(center, plane, R, r, split_param=100):
    u = np.linspace(0, 1, split_param)  
    v = np.linspace(0, 1, split_param)  
    u, v = np.meshgrid(u, v)

    u_spher = u * 2 * np.pi  
    v_spher = np.arcsin(2 * v - 1) 

    # Перевод точек в декартовую систему координат
    x = (R + r * np.cos(v_spher)) * np.cos(u_spher)
    y = (R + r * np.cos(v_spher)) * np.sin(u_spher)
    z = r * np.sin(v_spher)

    points = np.vstack([x.ravel(), y.ravel(), z.ravel()]).T

    normals = []
    for point in points:
        x, y, z = point - center  # Смещаем точку в локальную систему координат

        
        proj_length = np.sqrt(x**2 + y**2)
        proj_x = (x / proj_length) * R
        proj_y = (y / proj_length) * R
        proj_z = 0 

        normal_vector = np.array([x - proj_x, y - proj_y, z - proj_z])
        normal_vector /= np.linalg.norm(normal_vector)       
        normals.append(normal_vector)

    return points, np.array(normals)



In [6]:
def apply_constraints(points, normals, constraints):
    A = np.array([constraint[:3] for constraint in constraints])
    b = -np.array([constraint[3] for constraint in constraints])

    valid_points = []
    valid_normals = []
    for i, point in enumerate(points):
        if np.all(np.dot(A, point) >= b):  
            valid_points.append(point)
            valid_normals.append(normals[i])

    return np.array(valid_points), np.array(valid_normals)

In [7]:
def generate_points(surface, split_param=100, r_probe=1.0):
    if 'Atom_sur' in surface:
        center, radius, constraints = surface['Atom_sur'].values()
        points, normals = generate_surface_points(center, radius, split_param)
    elif 'Bond_sur' in surface:
        center, constraints = surface['Bond_sur'].values()
        points, normals = generate_surface_points(center, r_probe, split_param)
    elif 'Tor_sur' in surface:
        _, center, R, r, constraints = surface['Tor_sur'].values()
        points, normals = generate_toroidal_surface(center, R, r, split_param)
    else:
        raise ValueError("Неизвестный тип поверхности")
    
    return apply_constraints(points, normals, constraints)

In [10]:
surface_atom = {'Atom_sur': {'Center': (5,15,17), 'Radius': 10, 'Сonstraints': [[2,3,8,1],[1,1,2,2]]}}
surface_bond = {'Bond_sur': {'Center': (5,15,17), 'Сonstraints': [[2,3,8,1],[1,1,2,2]]}}
surface_tor = {'Tor_sur': {'Plane': [1,5,4,2], 'Center': (5,15,17), 'Radius_1': 20, 'Radius_2': 5, 'Сonstraints': [[2,3,8,1],[1,1,2,2]]}}

points_atom, normals_atom = generate_points(surface_atom)
points_bond, normals_bond = generate_points(surface_bond)
points_tor, normals_tor = generate_points(surface_tor)

print("Atom surface points:", points_atom[:5])  
print("Atom surface normals:", normals_atom[:5])
print("Bond surface points:", points_bond[:5])
print("Bond surface normals:", normals_bond[:5])
print("Torus surface points:", points_tor[:5])
print("Torus surface normals:", normals_tor[:5])

ValueError: too many values to unpack (expected 2)